# Laboratory Task 4

**Instruction:** Train a linear regression model in PyTorch using a regression dataset. Use the following parameters.

Criterion: MSE Loss <br>
Fully Connected Layers x 2 <br>
Batch Size: 8 <br>
Optimizer: SGD <br>
Epoch: 1000 <br>

In [1]:
# Import libraries

import torch
from sklearn import datasets
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

In [2]:
# Load diabetes dataset
diabetes = datasets.load_diabetes()
X, y = diabetes.data, diabetes.target

In [3]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Convert to Pytorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [5]:
# Create dataset and dataloader
train_ds = TensorDataset(X_train_tensor, y_train_tensor)
test_ds = TensorDataset(X_test_tensor, y_test_tensor)

train_dl = DataLoader(train_ds, batch_size=8, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=8)

In [6]:
# Define model with 2 layers
import torch.nn as nn

class LinearRegression(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LinearRegression, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Instantiate the model
input_size = X_train.shape[1] # Number of features in the diabetes dataset
hidden_size = 64 # Chosen number of hidden units
output_size = 1 # Predicting a single value (diabetes progression)

model = LinearRegression(input_size, hidden_size, output_size)

In [7]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [8]:
# Train the model
num_epochs = 1000

for epoch in range(num_epochs):
    for inputs, targets in train_dl:
        preds = model(inputs)
        loss = criterion(preds, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 13018.7002


Epoch [200/1000], Loss: 364.9501


Epoch [300/1000], Loss: 9606.5156


Epoch [400/1000], Loss: 534.8263


Epoch [500/1000], Loss: 1068.8887


Epoch [600/1000], Loss: 2225.7861


Epoch [700/1000], Loss: 22.8824


Epoch [800/1000], Loss: 2610.7632


Epoch [900/1000], Loss: 12.1712


Epoch [1000/1000], Loss: 0.7209


In [9]:
# Evaluate model

model.eval()

with torch.no_grad():
    total_test_loss = 0
    num_batches = 0
    for inputs, targets in test_dl:
        preds = model(inputs)
        loss = criterion(preds, targets)
        total_test_loss += loss.item()
        num_batches += 1

    average_test_loss = total_test_loss / num_batches

    print(f'Average Test Loss: {average_test_loss:.4f}')

Average Test Loss: 2575.2539
